In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization


2022-11-29 05:08:54.273477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 05:08:54.384423: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset ='/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Kerasmodel/aclImdb'

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

['README', 'train']

In [3]:
train_dir = '/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Kerasmodel/aclImdb/train/'
os.listdir(train_dir)

['pos']

In [5]:
batch_size = 8
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 281322 files belonging to 1 classes.
Using 225058 files for training.


2022-11-29 05:09:05.419781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 281322 files belonging to 1 classes.
Using 56264 files for validation.


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(10000, 20)

In [8]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.00443368, -0.00449153, -0.00977117, -0.03345424,  0.0241166 ,
         0.00446374, -0.0340552 , -0.02846661, -0.01939811, -0.02843589,
        -0.01712463,  0.04189447,  0.01391122, -0.0333804 ,  0.00141541,
        -0.022307  ,  0.03468401, -0.02461953, -0.03797468, -0.02145501],
       [-0.01741524, -0.03937967,  0.03784214, -0.03368684, -0.02853957,
         0.0419996 ,  0.02830886, -0.01529721,  0.0405785 , -0.04565322,
        -0.03636565,  0.04430708, -0.03249317, -0.0431648 , -0.02639714,
        -0.03605739, -0.03309214, -0.03815433, -0.01130884,  0.03706587],
       [-0.03984619,  0.00260301, -0.04776651, -0.008036  ,  0.03587971,
        -0.01534084,  0.01837516, -0.02610468, -0.01780371,  0.03342327,
        -0.04110683,  0.03456051,  0.02004584, -0.01930758,  0.00548307,
        -0.01961004,  0.03455678,  0.026465  , -0.02749038,  0.01312982]],
      dtype=float32)

In [9]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 20])

In [101]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 50000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [102]:
embedding_dim=100

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(10, activation='relu'),
  Dense(1)
])

In [103]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [104]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [105]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=7,
    callbacks=[tensorboard_callback])

Epoch 1/15
14515/14515 [==============================] - 363s 25ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 3.9553e-07 - val_accuracy: 1.0000
Epoch 2/15
14515/14515 [==============================] - 360s 25ms/step - loss: 2.2831e-08 - accuracy: 1.0000 - val_loss: 1.0467e-08 - val_accuracy: 1.0000
Epoch 3/15
14515/14515 [==============================] - 366s 25ms/step - loss: 4.1943e-10 - accuracy: 1.0000 - val_loss: 2.2265e-09 - val_accuracy: 1.0000
Epoch 4/15
14515/14515 [==============================] - 360s 25ms/step - loss: 1.1281e-10 - accuracy: 1.0000 - val_loss: 1.1729e-09 - val_accuracy: 1.0000
Epoch 5/15
14515/14515 [==============================] - 369s 25ms/step - loss: 5.9513e-11 - accuracy: 1.0000 - val_loss: 7.9318e-10 - val_accuracy: 1.0000
Epoch 6/15
14515/14515 [==============================] - 377s 26ms/step - loss: 3.9307e-11 - accuracy: 1.0000 - val_loss: 6.0040e-10 - val_accuracy: 1.0000
Epoch 7/15
14515/14515 [==============================] - 386s

In [111]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_5 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 100)          5000000   
                                                                 
 global_average_pooling1d_6   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_12 (Dense)            (None, 10)                1010      
                                                                 
 dense_13 (Dense)            (None, 1)                 11        
                                                                 
Total params: 5,001,021
Trainable params: 5,001,021
No

In [112]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 6864), started 3:02:01 ago. (Use '!kill 6864' to kill it.)

In [108]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [109]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  if word is not None and word !=' ' and word != '\t' and (not word.startswith(' ') )and (not word.startswith('\t')):
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()

In [110]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass